In [190]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats
import os
from keras.models import load_model

In [330]:
import cobra
%load_ext autoreload
import sys
if not '/home/nlarusstone/cf_fba' in sys.path:
    sys.path.append('/home/nlarusstone/cf_fba')
import src.utils as utils
import src.flux_utils as futils
import src.create_dataset as dataset
import src.flux_sample as fs
from src.get_test_data import get_test_data
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [182]:
for f in os.listdir('../results'):
    print f
    res_df = pd.read_csv('../results/' + f)
    #print res_df.head()
    res_df.drop('Unnamed: 0', inplace=True, axis=1)
    print scipy.stats.pearsonr(res_df.iloc[:, 0], res_df.iloc[:, 1])

echo_nls_unoptimized_results
(0.20672168021926396, 0.1455411499600651)
hand_nls_unoptimized_results
(-0.3087320241085889, 0.22793064480135675)
karim_karim_unoptimized_results
(-0.33548189565577985, 0.03201080584673351)
hand_nls_unoptimized_results_txtl
(-0.2966332397215686, 0.2476207415912054)
echo_nls_unoptimized_results_txtl
(0.3044916089198302, 0.029820032545076407)


In [332]:
latent_dim = 2
use_corr = True
froot = 'hand'
if froot == 'karim':
    dset = 'karim'
else:
    dset = 'nls'
txtl = False
resamp = True
layer_szs = [1024, 512, 256]

In [341]:
X_test.shape

(12500, 17, 2637)

In [343]:
encoder, generator, X_test, y_test, obj_col, cols, y_vals_d, test_enc, test_dec = get_test_data(
    froot, txtl, resamp, latent_dim, layer_szs, use_corr=True)

Load models ../models/encoder_epochs=200_batch=256_dimension=2_corr=True_scale=flux_zero_froot=hand_txtl=False_nlayers=3_resamp=True.h5
Read in data from ../data/hand_stacked_fluxes
Encoded data
Decoded data


In [344]:
model_f = '{0}_ecoli_cf_base{1}.sbml'.format(dset, '_txtl' if txtl else '')
test = '{0}_ecoli_cf_base{1}_0.sbml'.format(dset, '_txtl' if txtl else '')
if not os.path.exists('../models/{0}/{1}'.format(froot, test)):
    gen_models()
    
thresh = np.logspace(start=-2, stop=-1, num=20)
bad_rxns_t = []
res = []
for t in thresh:
    print t
    objs = []
    bad_rxns = []
    for idx, row in df.iterrows():
        model_f_i = '{0}_ecoli_cf_base{1}_{2}.sbml'.format(dset, '_txtl' if txtl else '', idx)
        model_i = cobra.io.read_sbml_model('../models/{0}/'.format(froot) + model_f_i)
        obj_name = str(model_i.objective.expression.args[0]).split('*')[1]
        dec_df = pd.DataFrame(data=test_dec[:, idx, :], columns=cols)
        bad_cols = cols[dec_df.mean() < t]
        bad_cols = bad_cols[bad_cols != obj_name]
        bad_rxns.append(bad_cols)
        model_i.remove_reactions(bad_cols)
        sol = model_i.optimize()
        objs.append(sol.objective_value)
    bad_rxns_t.append(bad_rxns)
    print scipy.stats.pearsonr(objs, df['OUT'])
    res.append(objs)

0.01
(0.2576290968199518, 0.31813030708810797)
0.011288378916846888
(0.43357909048256066, 0.08208518674168952)
0.012742749857031334
(0.4335790904825604, 0.08208518674168974)
0.01438449888287663
(0.4335790904825604, 0.08208518674168974)
0.016237767391887217
(-0.5138068419962464, 0.034878785341557465)
0.018329807108324356
(0.4601567049045092, 0.0630828916135493)
0.0206913808111479
(0.21881232948268128, 0.3988021536655779)
0.023357214690901226
(0.42382506399567393, 0.09000482405047663)
0.026366508987303583
(nan, 1.0)
0.029763514416313176


IOError: Error reading file '../models/hand/nls_ecoli_cf_base_15.sbml': failed to load external entity "../models/hand/nls_ecoli_cf_base_15.sbml"

In [199]:
rxn_amt = 5
addl_amt = 1
batch_size = 50
n_batches = batch_size / rxn_amt
model_f = '{0}_ecoli_cf_base{1}.sbml'.format('nls', '_txtl' if txtl else '')

print 'Read in data'
df = fs.get_exp_data(froot)
#df.drop(columns=['Area_1', 'Area_2', 'Conc_1', 'Conc_2'], inplace=True)

cfps_conc = pd.read_csv('../data/{0}_concs'.format('nls'), index_col='compound')
if not froot == 'karim':
    cfps_conc.drop('final_conc', inplace=True, axis=1)
    nrg_mix = pd.read_csv('../data/energy_mix.csv', index_col='compound')
else:
    nrg_mix = None
#print cfps_conc

print 'Time to generate model specific conditions'
model = cobra.io.read_sbml_model('../models/{0}'.format(model_f))
print 'Model {0} read in'.format(model_f)

Read in data
(1591, 38)
          start_conc    amt
compound                   
pi          0.049038  0.100
mg2         0.400000  0.074
k           3.000000  0.080
nad         0.004540  0.500
coa         0.003530  0.500
camp        0.010127  0.500
5fthf       0.000921  0.500
spmd        0.001808  0.500
glc__D      0.054190  0.500
atp         0.020931  0.500
gtp         0.020968  0.500
ctp         0.012606  0.500
utp         0.012601  0.500
trna        0.000104  0.500
ala         0.178191  1.000
arg         0.215287  1.000
asn         0.178200  1.000
asp         0.178193  1.000
cys         0.178022  1.000
glu         0.178198  1.000
gln         0.178026  1.000
gly         0.178205  1.000
his         0.178192  1.000
ile         0.178207  1.000
leu         0.178029  1.000
lys         0.178022  1.000
met         0.178017  1.000
phe         0.178198  1.000
pro         0.178196  1.000
ser         0.179566  1.000
thr         0.178198  1.000
trp         0.178201  1.000
tyr         0.178020  1.

NameError: name 'i' is not defined

In [225]:
t = 10 ** -4.58
pd.DataFrame(data=test_dec[:, 0, :], columns=cols)
bad_cols = cols[dec_df.mean() < t]
bad_cols

array(['EX_23camp_e', 'EX_23ccmp_e', 'EX_23cump_e', 'EX_23dappa_e',
       'EX_34dhpac_e', 'EX_3hcinnm_e', 'EX_3hpppn_e', 'EX_acac_e',
       'EX_acgal_e', 'EX_acmum_e', 'EX_acnam_e', 'EX_acser_e',
       'EX_all__D_e', 'EX_arbt_e', 'EX_ascb__L_e', 'EX_aso3_e',
       'EX_but_e', 'EX_cd2_e', 'EX_cgly_e', 'EX_cm_e', 'EX_crn__D_e',
       'EX_cu_e', 'EX_cynt_e', 'EX_doxrbcn_e', 'EX_fe3dhbzs_e',
       'EX_fe3hox_e', 'EX_fecrm_un_e', 'EX_feoxam_un_e', 'EX_fusa_e',
       'EX_g3pi_e', 'EX_gal_bD_e', 'EX_galur_e', 'EX_glcr_e',
       'EX_glcur_e', 'EX_glyald_e', 'EX_gthrd_e', 'EX_lcts_e',
       'EX_mal__D_e', 'EX_maltpt_e', 'EX_malttr_e', 'EX_manglyc_e',
       'EX_mincyc_e', 'EX_mnl_e', 'EX_novbcn_e', 'EX_ppal_e', 'EX_pppn_e',
       'EX_ppt_e', 'EX_progly_e', 'EX_psclys_e', 'EX_rfamp_e',
       'EX_sbt__D_e', 'EX_so2_e', 'EX_tma_e', 'EX_tungs_e', 'EX_tyrp_e',
       'EX_udpacgal_e', 'EX_udpgal_e', '23PDE2pp', '23PDE7pp', '23PDE9pp',
       '2DGULRGx', '2DGULRx', '3HCINNMH', '3KGK', '42A1

In [238]:
res = []
thresh = np.logspace(start=-2, stop=-1, num=20)
for t in thresh:
    print t
    objs = []
    for idx, row in df.iterrows():
        print idx
        cfps_conc_tmp = cfps_conc.copy()
        cfps_conc_tmp = fs.update_vals(cfps_conc_tmp, row, n_batches, nrg_mix=nrg_mix, replace=args.froot == 'karim')
        tot_rxn_amt = rxn_amt + addl_amt
        if args.froot == 'karim':
            exp_concs = cfps_conc_tmp['final_conc']
        else:
            exp_concs = fs.conc_dilution(cfps_conc_tmp['start_conc'], (rxn_amt / 5.0) * cfps_conc_tmp['amt'], tot_rxn_amt)
        model_i = fs.change_conc(model, exp_concs)
        dec_df = pd.DataFrame(data=test_dec[:, idx, :], columns=cols)
        bad_cols = cols[dec_df.mean() < t]
        bad_cols = bad_cols[bad_cols != 'DM_PROTEIN_RFP']
        model_i.remove_reactions(bad_cols)
        #print model_i.reactions.EX_mg2_c.bounds
        #print model_i.reactions.EX_pi_c.bounds
        sol = model_i.optimize()
        objs.append(sol.objective_value)
    print scipy.stats.pearsonr(objs, df['OUT'])
    res.append(objs)
#res_df = pd.DataFrame(res)
#res_df['Actual'] = df['OUT']
#res_df.to_csv('../results/{0}_{1}_unoptimized_results{2}'.format(args.froot, args.dataset, '_txtl' if args.txtl else ''))

0.01
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.4986358014655035, 0.04161178360521059)
0.011288378916846888
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.28967362750165354, 0.2594151759116899)
0.012742749857031334
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.2837875229029313, 0.2696575023747873)
0.01438449888287663
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.26775175958856107, 0.2988018711529221)
0.016237767391887217
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.49201106510706677, 0.04484266070960983)
0.018329807108324356
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.38050067052858655, 0.13188938826130858)
0.0206913808111479
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.24732639648539045, 0.33853636394595665)
0.023357214690901226
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.24536361763590187, 0.3425073142890563)
0.026366508987303583
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.36745302444130973, 0.14678387099171755)
0.029763514416313176
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.38822

In [231]:
for r in res:
    print r
    print scipy.stats.pearsonr(r, df['OUT'])

[0.0, 0.0, 0.0, 0.0013010358804909973, 0.0012985748293459635, 0.0, 0.0, 0.0013016592785199692, 0.0, 0.0, 0.0012971660257889725, 0.0012972470295384906, 0.0013009548767414798, 0.0, 0.0, 0.0, 0.0013009953786162385]
(0.08155369196996047, 0.7556785156170105)
[0.0, 0.0, 0.0, 0.0, 0.0012985748293459635, 0.0, 0.0, 0.0013016592785199692, 0.0, 0.0, 0.0012971660257889775, 0.0012972470295384915, 0.0013009548767414808, 0.0, 0.0, 0.0, 0.0013009953786162385]
(-0.1297391306149426, 0.619690441557516)
[0.0, 0.0, 0.0, 0.0, 0.0012985748293459622, 0.0, 0.0, 0.0013016592785199692, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013009953786162576]
(0.3148378162382709, 0.21838487603471352)
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013016592785199692, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013009953786162437]
(0.1930383558923267, 0.4578945896045695)
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0013016592785199668, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
(0.46015670490450916, 0.0630828916135493)


In [246]:
#t = 0.016237767391887217
t = 0.01
bad_rxns = []
objs = []
for idx, row in df.iterrows():
    print idx
    cfps_conc_tmp = cfps_conc.copy()
    cfps_conc_tmp = fs.update_vals(cfps_conc_tmp, row, n_batches, nrg_mix=nrg_mix, replace=args.froot == 'karim')
    tot_rxn_amt = rxn_amt + addl_amt
    if args.froot == 'karim':
        exp_concs = cfps_conc_tmp['final_conc']
    else:
        exp_concs = fs.conc_dilution(cfps_conc_tmp['start_conc'], (rxn_amt / 5.0) * cfps_conc_tmp['amt'], tot_rxn_amt)
    model_i = fs.change_conc(model, exp_concs)
    dec_df = pd.DataFrame(data=test_dec[:, idx, :], columns=cols)
    bad_cols = cols[dec_df.mean() < t]
    bad_cols = bad_cols[bad_cols != 'DM_PROTEIN_RFP']
    bad_rxns.append(bad_cols)
    model_i.remove_reactions(bad_cols)
    #print model_i.reactions.EX_mg2_c.bounds
    #print model_i.reactions.EX_pi_c.bounds
    sol = model_i.optimize()
    objs.append(sol.objective_value)
print scipy.stats.pearsonr(objs, df['OUT'])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
(-0.4986358014655035, 0.04161178360521059)


In [261]:
prot = model.reactions

In [264]:
brs = bad_rxns[0]
prot = model.reactions
for br in bad_rxns:
    print len(br)
    for r in br:
        if r in prot:
            prot.remove(r)
    brs = np.intersect1d(brs, br)

550
543
512
551
524
529
512
513
543
503
522
500
506
517
504
512
470


In [267]:
prot

[<Reaction DM_4crsol_c at 0xdffdb950>,
 <Reaction DM_oxam_c at 0xdffdba90>,
 <Reaction EX_12ppd__S_e at 0xdffdbc10>,
 <Reaction EX_15dap_e at 0xdffdbcd0>,
 <Reaction EX_ac_e at 0xdffde4d0>,
 <Reaction EX_ade_e at 0xdffde8d0>,
 <Reaction EX_alltn_e at 0xdffdec10>,
 <Reaction EX_ca2_e at 0xdffe4050>,
 <Reaction EX_cbl1_e at 0xdffe40d0>,
 <Reaction EX_cit_e at 0xdffe4250>,
 <Reaction EX_cl_e at 0xdffe4290>,
 <Reaction EX_co2_e at 0xdffe4390>,
 <Reaction EX_cobalt2_e at 0xdffe43d0>,
 <Reaction EX_cu2_e at 0xdffe4690>,
 <Reaction EX_dcmp_e at 0xdffe48d0>,
 <Reaction EX_enlipa_e at 0xdffe4ed0>,
 <Reaction EX_fe2_e at 0xdffeb1d0>,
 <Reaction EX_fe3_e at 0xdffeb210>,
 <Reaction EX_feenter_e at 0xdffeb490>,
 <Reaction EX_for_e at 0xdffeb5d0>,
 <Reaction EX_fru_e at 0xdffeb610>,
 <Reaction EX_g1p_e at 0xdffeb850>,
 <Reaction EX_galct__D_e at 0xdffebb50>,
 <Reaction EX_gly_e at 0xe00b20d0>,
 <Reaction EX_glyc2p_e at 0xe00b22d0>,
 <Reaction EX_gthox_e at 0xe00b2410>,
 <Reaction EX_h_e at 0xe00b24d

In [269]:
m = cobra.Model('cfps')

In [270]:
m.add_reactions(prot)

In [271]:
m.objective = m.reactions.DM_PROTEIN_RFP

In [273]:
m.optimize()
m.summary()

IN FLUXES    OUT FLUXES    OBJECTIVES
-----------  ------------  -----------------
h_c  3.85                  DM_PROTEIN_RFP  0
